In [36]:
import numpy as np
from matplotlib import pyplot as plt
from ruamel.yaml import YAML

yaml=YAML() 

In [30]:
fil = open('../../data/reaction_mechanisms/zahnle_earth.yaml','r')
zahnle = yaml.load(fil)
fil.close()
zahnle_species = []
for i in range(len(zahnle['species'])):
    zahnle_species.append(zahnle['species'][i]['name'])

In [31]:
import bibtexparser

with open('henry.bib') as bibtex_file:
    bib_database = bibtexparser.load(bibtex_file)

ID = {}
for entry in bib_database.entries:
    ID[entry['ID']] = float(entry['year'])

In [32]:
fil = open('HbpSI.f90','r')
lines = fil.readlines()
fil.close()

In [38]:
inds = []
for i in range(len(lines)):
    if lines[i][0:10] == '! species:':
        inds.append(i)

        
henry = []
        
for i in range(len(inds)-1):
    data = lines[inds[i]:inds[i+1]-1]
    
    species = data[1].split()[2].replace('_','').replace('{','').replace('}','')
    
    casrn = data[3].split()[2]
    
    refs = []
    for j in range(5,len(data)):
        line = data[j]
        if "EXP" in line:
            try:
                ind = line.index('ref:')
                refs.append([line[ind:].split()[1],j])
            except:
                pass
    if len(refs) > 0:
        
        
        years = []
        for ref in refs:
            years.append(ID[ref[0]])
        ind = np.argmax(years)
        ind_best = refs[ind][1]
        
        line = data[ind_best]
#         print(line)
        A = float(line.split('=')[1].split('*')[0])
        B = float(line.split('EXP(')[1].split('*')[0])
        
        henry.append({})
        henry[-1]['name'] = species
#         henry[-1]['cas-rn'] = casrn
        henry[-1]['A'] = A
        henry[-1]['B'] = B
        

    else:
        pass
            
#     break
    
    
    

In [39]:
photo_henry = []

for sp in zahnle_species:
    for entry in henry:
        if entry['name'] == sp:
            photo_henry.append(entry)
            break

In [40]:
fil = open('henry.yaml','w')
yaml.dump(photo_henry,fil)
fil.close()

In [41]:
len(photo_henry)

35